In [2]:
!pip install git+https://github.com/openai/CLIP.git



  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-6n9lekho
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-6n9lekho
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [16]:
def fine_tune_clip(clip_model, train_loader, val_loader, classnames, num_epochs=10):
    # 1. Model Preparation
    # Freeze all layers except the projection layers
    for name, param in clip_model.named_parameters():
        if 'visual.proj' not in name and 'text_projection' not in name:
            param.requires_grad = False
        else:
            print(f"Training layer: {name}")

    # 2. Optimizer Configuration
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, clip_model.parameters()),
        lr=1e-6,  # Very conservative learning rate
        betas=(0.9, 0.98),
        eps=1e-6,
        weight_decay=0.01
    )
    
    # 3. Learning Rate Scheduling with Warmup
    warmup_steps = 500
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer,
        lambda step: min(1.0, step / warmup_steps)
    )
    
    criterion = torch.nn.CrossEntropyLoss()
    
    # 4. Text Input Preparation
    with torch.no_grad():
        text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in classnames]).to(device)
        text_features = clip_model.encode_text(text_inputs)
        text_features = text_features / text_features.norm(dim=1, keepdim=True)
    
    # 5. CLIP Normalization Parameters
    clip_mean = torch.tensor([0.48145466, 0.4578275, 0.40821073], device=device).view(1, 3, 1, 1)
    clip_std = torch.tensor([0.26862954, 0.26130258, 0.27577711], device=device).view(1, 3, 1, 1)
    
    # 6. Training Loop with Stability Checks
    best_acc = 0
    
    for epoch in range(num_epochs):
        clip_model.train()
        train_loss = 0
        grad_norms = []
        
        for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
            images, labels = images.to(device), labels.to(device)
            
            # Apply CLIP normalization with numerical stability checks
            images = (images - clip_mean) / clip_std
            if torch.isnan(images).any():
                print("NaN detected in images after normalization!")
                continue
            
            # Forward pass with stability checks
            optimizer.zero_grad()
            
            image_features = clip_model.encode_image(images)
            if torch.isnan(image_features).any():
                print("NaN in image features!")
                continue
                
            image_features = image_features / image_features.norm(dim=1, keepdim=True)
            
            # Use pre-computed text features
            logit_scale = clip_model.logit_scale.exp().clamp(max=100)  # Clamp to prevent overflow
            logits = logit_scale * image_features @ text_features.T
            
            if torch.isnan(logits).any():
                print(f"NaN in logits! Scale: {logit_scale.item()}")
                continue
                
            loss = criterion(logits, labels)
            
            if torch.isnan(loss):
                print("NaN loss detected!")
                continue
                
            # Backward pass with gradient monitoring
            loss.backward()
            
            # Gradient clipping and monitoring
            grad_norm = torch.nn.utils.clip_grad_norm_(
                filter(lambda p: p.requires_grad, clip_model.parameters()),
                max_norm=1.0
            )
            grad_norms.append(grad_norm.item())
            
            if torch.isnan(grad_norm):
                print("NaN gradients detected!")
                optimizer.zero_grad()
                continue
                
            optimizer.step()
            scheduler.step()
            
            train_loss += loss.item()
            
            # Debug prints every 100 batches
            if batch_idx % 100 == 0:
                print(f"\nBatch {batch_idx}:")
                print(f"  Loss: {loss.item():.4f}")
                print(f"  Grad norm: {grad_norm:.4f}")
                print(f"  Logit scale: {logit_scale.item():.4f}")
                print(f"  Max logit: {logits.max().item():.4f}")
                print(f"  Min logit: {logits.min().item():.4f}")
        
        avg_train_loss = train_loss / len(train_loader)
        avg_grad_norm = sum(grad_norms) / len(grad_norms)
        print(f"\nEpoch {epoch+1}:")
        print(f"  Avg Train Loss: {avg_train_loss:.4f}")
        print(f"  Avg Grad Norm: {avg_grad_norm:.4f}")
        
        # Validation
        val_acc = validate_clip(clip_model, val_loader, text_features)
        print(f"  Val Acc: {val_acc:.2f}%")
        
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(clip_model.state_dict(), "best_clip_model.pth")
    
    return clip_model

def validate_clip(clip_model, val_loader, text_features):
    clip_model.eval()
    correct = 0
    total = 0
    
    clip_mean = torch.tensor([0.48145466, 0.4578275, 0.40821073], device=device).view(1, 3, 1, 1)
    clip_std = torch.tensor([0.26862954, 0.26130258, 0.27577711], device=device).view(1, 3, 1, 1)
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            images = (images - clip_mean) / clip_std
            
            image_features = clip_model.encode_image(images)
            image_features = image_features / image_features.norm(dim=1, keepdim=True)
            
            logit_scale = clip_model.logit_scale.exp().clamp(max=100)
            logits = logit_scale * image_features @ text_features.T
            
            _, predicted = torch.max(logits, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    return 100 * correct / total

In [9]:
def fine_tune_blip(blip_model, blip_processor, train_loader, num_epochs=3, classnames):
    optimizer = torch.optim.AdamW(blip_model.parameters(), lr=5e-5)
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        print("Blip epoch:", epoch, num_epochs)
        blip_model.train()
        epoch_loss = 0
        
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images = images.to(device)
            
            # Convert to PIL images
            pil_images = []
            for img in images:
                np_img = img.cpu().numpy().transpose(1, 2, 0)
                np_img = np.clip(np_img, 0, 1)
                pil_img = Image.fromarray((np_img * 255).astype(np.uint8))
                pil_images.append(pil_img)
            
            # Create inputs - using classnames as prompts
            inputs = blip_processor(
                images=pil_images, 
                text=[f"a photo of a {classnames[l]}" for l in labels],
                return_tensors="pt",
                padding=True,
                truncation=True
            ).to(device)
            
            # Forward pass
            outputs = blip_model(**inputs)
            loss = outputs.loss
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")
        
        if avg_loss < best_loss:
            best_loss = avg_loss
            torch.save(blip_model.state_dict(), "best_blip_model.pth")
    
    return blip_model

In [17]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import clip
from transformers import BlipProcessor, BlipForConditionalGeneration
import numpy as np
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class TinyImageNet(Dataset):
    def __init__(self, root, split='val', transform=None):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.split = split
        
        if split == 'train':
            self.image_paths = []
            self.labels = []
            
            # Process train directory which has subdirectories by class
            train_dir = os.path.join(self.root, 'train')
            self.classes = sorted(os.listdir(train_dir))
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
            
            for class_name in self.classes:
                class_dir = os.path.join(train_dir, class_name, 'images')
                if not os.path.isdir(class_dir):
                    continue
                    
                for img_name in os.listdir(class_dir):
                    if img_name.endswith('.JPEG'):
                        self.image_paths.append(os.path.join(class_dir, img_name))
                        self.labels.append(self.class_to_idx[class_name])
                        
        elif split == 'val':
            self.image_paths = []
            self.labels = []
            
            # Process val directory
            val_dir = os.path.join(self.root, 'val')
            images_dir = os.path.join(val_dir, 'images')
            
            # Read val annotations
            val_annotations_file = os.path.join(val_dir, 'val_annotations.txt')
            self.classes = []
            self.class_to_idx = {}
            
            with open(val_annotations_file, 'r') as f:
                for line in f:
                    parts = line.strip().split('\t')
                    img_name, class_id = parts[0], parts[1]
                    
                    if class_id not in self.class_to_idx:
                        self.classes.append(class_id)
                        self.class_to_idx[class_id] = len(self.classes) - 1
                    
                    self.image_paths.append(os.path.join(images_dir, img_name))
                    self.labels.append(self.class_to_idx[class_id])

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
            
        return img, label

def get_classnames(dataset, words_file_path):
    """Get formatted class names from dataset using WordNet mappings."""
    if hasattr(dataset, 'classes'):
        # Load WordNet ID to name mapping
        wordnet_map = {}
        with open(words_file_path, 'r') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) == 2:
                    wordnet_id, names = parts
                    # Take the first name if there are multiple comma-separated ones
                    name = names.split(',')[0].strip()
                    wordnet_map[wordnet_id] = name
        
        # Replace WordNet IDs with human-readable names
        classnames = []
        for c in dataset.classes:
            if c in wordnet_map:
                classnames.append(wordnet_map[c])
            else:
                # Try without 'n' prefix if not found directly
                if c.startswith('n') and c[1:] in wordnet_map:
                    classnames.append(wordnet_map[c[1:]])
                else:
                    # Fallback to the original ID
                    classnames.append(c)
        
        return classnames
    return None

def prepare_dataset(root_dir):
    """Prepare and return the Tiny ImageNet dataset with appropriate transforms."""
    # Basic transform without normalization for dataset loading
    basic_transform = transforms.Compose([
        transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    
    # Create dataset instance
    val_dataset = TinyImageNet(root_dir, split='val', transform=basic_transform)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
    
    return val_dataset, val_loader
    
def generate_captions(blip_model, blip_processor, images, prompts):
    """Generate captions for the given images using BLIP model."""
    all_captions = []
    
    # Create PIL images from tensors
    pil_images = []
    for img in images:
        # Convert tensor to PIL image
        # Ensure values are within 0-1 range
        np_img = img.cpu().numpy().transpose(1, 2, 0)
        np_img = np.clip(np_img, 0, 1)
        pil_img = Image.fromarray((np_img * 255).astype(np.uint8))
        pil_images.append(pil_img)
    
    # Process images with BLIP
    batch_size = len(pil_images)
    with torch.no_grad():
        for prompt in prompts:
            batch_captions = []
            
            # Process each image individually
            for i in range(batch_size):
                inputs = blip_processor(images=pil_images[i], text=prompt, return_tensors="pt").to(device)
                generated_ids = blip_model.generate(**inputs, max_length=50)
                caption = blip_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
                batch_captions.append(caption)
                
            all_captions.append(batch_captions)
    
    return all_captions


def compute_image_embeddings(clip_model, images):
    """Compute CLIP image embeddings with proper normalization."""
    # Apply CLIP-specific normalization
    clip_mean = torch.tensor((0.48145466, 0.4578275, 0.40821073), device=images.device).view(1, 3, 1, 1)
    clip_std = torch.tensor((0.26862954, 0.26130258, 0.27577711), device=images.device).view(1, 3, 1, 1)
    normalized_images = (images - clip_mean) / clip_std
    
    with torch.no_grad():
        image_features = clip_model.encode_image(normalized_images)
        image_features = image_features / image_features.norm(dim=1, keepdim=True)
    return image_features

def compute_text_embeddings(clip_model, texts):
    """Compute CLIP text embeddings."""
    with torch.no_grad():
        text_features = clip_model.encode_text(texts)
        text_features = text_features / text_features.norm(dim=1, keepdim=True)
    return text_features

def image_caption_encoding(image_probs, caption_probs, K=5, xi=0.08, epsilon=1e-12):
    """
    Implement ICE method as described in the paper.
    
    Args:
        image_probs: Probability distribution from image embeddings
        caption_probs: Probability distribution from caption embeddings
        K: Number of top classes to consider
        xi: Scaling factor for lambda
        epsilon: Small constant to avoid division by zero
        
    Returns:
        Final probability distribution after ICE
    """
    # Find top K classes based on image probabilities
    topk_values, topk_indices = torch.topk(image_probs, k=K, dim=1)
    
    # Extract probabilities for top K classes
    image_topk_probs = torch.gather(image_probs, 1, topk_indices)
    caption_topk_probs = torch.gather(caption_probs, 1, topk_indices)
    
    # Compute standard deviation for confidence selection
    image_std = torch.std(image_topk_probs, dim=1, keepdim=True)
    caption_std = torch.std(caption_topk_probs, dim=1, keepdim=True)
    
    # Compute lambda based on equation 3 in the paper
    norm = torch.maximum(torch.norm(torch.cat([image_std, caption_std], dim=1), dim=1, keepdim=True), 
                         torch.tensor(epsilon, device=device))
    
    lambda_values = xi * (caption_std / norm)
    
    # Combine image and caption probabilities
    combined_topk_probs = image_topk_probs + lambda_values * caption_topk_probs
    
    # Create new probability distribution
    output_probs = torch.zeros_like(image_probs)
    output_probs.scatter_(1, topk_indices, combined_topk_probs)
    
    return output_probs

def evaluate_model(clip_model, blip_model, blip_processor, val_loader, classnames, caption_prompts):
    """Evaluate the model on the validation set."""
    text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in classnames]).to(device)
    text_features = compute_text_embeddings(clip_model, text_inputs)
    
    total = 0
    correct_base = 0
    correct_ice = 0
    
    # Print sample outputs for the first few batches only
    print_samples = 200
    sample_count = 0
    
    for batch_idx, (images, labels) in enumerate(tqdm(val_loader)):
        images = images.to(device)
        labels = labels.to(device)
        batch_size = images.shape[0]
        total += batch_size
        
        # Compute image embeddings and probabilities
        image_features = compute_image_embeddings(clip_model, images)
        image_logits = 100.0 * image_features @ text_features.T
        image_probs = F.softmax(image_logits, dim=1)
        
        # Generate captions using BLIP
        batch_captions = generate_captions(blip_model, blip_processor, images, caption_prompts)
        
        # Process captions
        caption_features_list = []
        for prompt_captions in batch_captions:
            caption_tokens = clip.tokenize(prompt_captions).to(device)
            caption_features = compute_text_embeddings(clip_model, caption_tokens)
            caption_features_list.append(caption_features)
        
        # Compute centroid of caption embeddings
        caption_features = torch.stack(caption_features_list).mean(dim=0)
        
        # Compute caption probabilities
        caption_logits = 100.0 * caption_features @ text_features.T
        caption_probs = F.softmax(caption_logits, dim=1)
        
        # Apply ICE
        ice_probs = image_caption_encoding(image_probs, caption_probs)
        
        # Compute predictions
        _, base_preds = torch.max(image_probs, 1)
        _, ice_preds = torch.max(ice_probs, 1)
        
        # Update metrics
        correct_base += (base_preds == labels).sum().item()
        correct_ice += (ice_preds == labels).sum().item()
        
        # Print examples for debugging
        if batch_idx < print_samples:
            print(f"\n--- Batch {batch_idx} examples ---")
            # Print top 5 examples from the batch
            for i in range(min(5, batch_size)):
                true_label = labels[i].item()
                base_pred = base_preds[i].item()
                ice_pred = ice_preds[i].item()
                
                print(f"\nImage {sample_count + i}:")
                print(f"  True class: {classnames[true_label]}")
                print(f"  CLIP prediction: {classnames[base_pred]}")
                print(f"  ICE prediction: {classnames[ice_pred]}")
                
                # Print captions from all prompts
                print("  Captions:")
                for j, prompt in enumerate(caption_prompts):
                    print(f"    {prompt}: {batch_captions[j][i]}")
                
                # Print confidence scores for predictions
                base_conf = image_probs[i, base_pred].item() * 100
                ice_conf = ice_probs[i, ice_pred].item() * 100
                print(f"  CLIP confidence: {base_conf:.2f}%")
                print(f"  ICE confidence: {ice_conf:.2f}%")
            
            sample_count += batch_size
    
    base_acc = 100 * correct_base / total
    ice_acc = 100 * correct_ice / total
    
    return base_acc, ice_acc

def main():
    tiny_imagenet_path = '/kaggle/input/tiny-imagenet/tiny-imagenet-200'

    
    # Prepare datasets
    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    
    val_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    
    train_dataset = TinyImageNet(tiny_imagenet_path, split='train', transform=train_transform)
    val_dataset = TinyImageNet(tiny_imagenet_path, split='val', transform=val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)
        
    # Load models
    clip_model, _ = clip.load("ViT-B/32", device=device)
    blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
    classnames = get_classnames(val_dataset, '/kaggle/input/tiny-imagenet/tiny-imagenet-200/words.txt')

    # Fine-tune models
    print("Fine-tuning CLIP...")
    clip_model = fine_tune_clip(clip_model, train_loader, val_loader, classnames)
    
    # Evaluate with fine-tuned models
    caption_prompts = # Dataset-specific prompts for BLIP
caption_prompts = [
    "Describe the specific object in this image for fine-grained classification",
    "What is the main object in this image? Include details like color, shape, and texture",
    "Identify this object precisely"
]
    
    print("\nEvaluating with fine-tuned models...")
    base_acc, ice_acc = evaluate_model(
        clip_model, blip_model, blip_processor, val_loader, classnames, caption_prompts
    )
    
    print(f"Base CLIP Accuracy: {base_acc:.2f}%")
    print(f"ICE Accuracy: {ice_acc:.2f}%")
    print(f"Improvement: {ice_acc - base_acc:.2f}%")
if __name__ == "__main__":
    main()

Using device: cuda
Fine-tuning CLIP...
Training layer: text_projection
Training layer: visual.proj


Epoch 1:   0%|          | 0/1563 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL


Batch 0:
  Loss: 8.2031
  Grad norm: 24.5781
  Logit scale: 100.0000
  Max logit: 32.6875
  Min logit: 9.2578


Epoch 1:   7%|▋         | 103/1563 [00:09<02:03, 11.85it/s]


Batch 100:
  Loss: 8.2188
  Grad norm: 24.8750
  Logit scale: 100.0000
  Max logit: 31.8750
  Min logit: 9.6953


...


--- Batch 154 examples ---

Image 9856:
  True class: oboe
  CLIP prediction: oboe
  ICE prediction: oboe
  Captions:
    a photo of: a photo of a woman playing a drum
    describe specifically: describe specifically, the person is a person
    be concise: be concise - live at the royal albert park, toronto
  CLIP confidence: 26.00%
  ICE confidence: 26.00%

Image 9857:
  True class: spider web
  CLIP prediction: fountain
  ICE prediction: fountain
  Captions:
    a photo of: a photo of a black and white image of a black and white image of a black and white image of a black and white image of a
    describe specifically: describe specifically, the stars are not in the same plane
    be concise: be concisement of the universe
  CLIP confidence: 4.14%
  ICE confidence: 4.14%

Image 9858:
  True class: computer keyboard
  CLIP prediction: computer keyboard
  ICE prediction: computer keyboard
  Captions:
    a photo of: a photo of a city at night
    describe specifically: describe specif

 99%|█████████▉| 156/157 [1:35:14<00:36, 36.97s/it]


--- Batch 155 examples ---

Image 9920:
  True class: flagpole
  CLIP prediction: flagpole
  ICE prediction: flagpole
  Captions:
    a photo of: a photo of a kite flying in the sky
    describe specifically: describe specifically, the wind is the wind
    be concise: be concisement of the sun
  CLIP confidence: 87.99%
  ICE confidence: 87.99%

Image 9921:
  True class: black widow
  CLIP prediction: black widow
  ICE prediction: black widow
  Captions:
    a photo of: a photo of a black and white dog
    describe specifically: describe specifically, the term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term term
    be concise: be concisement - a new study of the human brain
  CLIP confidence: 73.54%
  ICE confidence: 73.54%

Image 9922:
  True class: steel arch bridge
  CLIP prediction: steel arch bridge
  ICE prediction

100%|██████████| 157/157 [1:35:23<00:00, 36.46s/it]


--- Batch 156 examples ---

Image 9984:
  True class: chain
  CLIP prediction: sea cucumber
  ICE prediction: sea cucumber
  Captions:
    a photo of: a photo of a man in a wet suit
    describe specifically: describe specifically, the bear is a member of the bear family
    be concise: be concisely, the best way to get out of the water
  CLIP confidence: 11.71%
  ICE confidence: 11.71%

Image 9985:
  True class: candle
  CLIP prediction: candle
  ICE prediction: candle
  Captions:
    a photo of: a photo of a cake with candles on it
    describe specifically: describe specifically, the candle is a symbol of the candlelighter ' s life
    be concise: be concisement - a candle and a candle
  CLIP confidence: 71.88%
  ICE confidence: 78.47%

Image 9986:
  True class: jellyfish
  CLIP prediction: jellyfish
  ICE prediction: jellyfish
  Captions:
    a photo of: a photo of a blue jellyfish in the ocean
    describe specifically: describe specifically, the earth is a planet
    be concise: